---

# Training Multilayer perception (Geotag - Latent space mapping)

>This notebook contains code for training an MLP to control the latent space using geotags.
>Latent vectors are generated with audio files using the encoder.
>Their corresponding geotags are used as input and latent vectors as output to train the model.

---

In [1]:
import pandas as pd
import numpy as np
import librosa
import tensorflow as tf
from tensorflow.keras import layers, models
import IPython.display as ipd
from tensorflow.keras.models import load_model

## Dataset

### Load audio file path-geotag mappings

In [2]:
df = pd.read_csv('../corpus/mappings.csv')

In [3]:
df.head()

,path,lat,long
0,../corpus/audio/169884.mp3,0.501047,0.041804
1,../corpus/audio/169885.mp3,0.501052,0.041806
2,../corpus/audio/697381.mp3,0.187898,0.534158
3,../corpus/audio/187893.mp3,0.459852,0.012109
4,../corpus/audio/788102.mp3,0.236748,-0.876991


### Get input geotag

In [6]:
X = df[['lat', 'long']].to_numpy()

### Load latent vectors

In [8]:
latents = np.load('../models/latents.npy')

## Multilayer perceptron

In [32]:
mlp = tf.keras.Sequential()
mlp.add(layers.Input(shape=(2,)))
mlp.add(layers.Dense(64, activation='relu'))
mlp.add(layers.Dense(128, activation='relu'))
mlp.add(layers.Dense(256, activation='relu'))
mlp.add(layers.Dense(128, activation='relu'))
mlp.add(layers.Dense(64, activation='relu'))

In [33]:
mlp.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 64)                192       
                                                                 
 dense_17 (Dense)            (None, 128)               8320      
                                                                 
 dense_18 (Dense)            (None, 256)               33024     
                                                                 
 dense_19 (Dense)            (None, 128)               32896     
                                                                 
 dense_20 (Dense)            (None, 64)                8256      
                                                                 
Total params: 82,688
Trainable params: 82,688
Non-trainable params: 0
_________________________________________________________________


In [41]:
mlp.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='mean_squared_error')

In [42]:
mlp.fit(X, latents, epochs=5000, verbose=False, batch_size=latents.shape[0])
mlp.fit(X, latents, epochs=50, verbose=True, batch_size=latents.shape[0])

Epoch 1/50
1/1 [==============================] - 0s 8ms/step - loss: 0.9782
Epoch 2/50
1/1 [==============================] - 0s 7ms/step - loss: 0.9782
Epoch 3/50
1/1 [==============================] - 0s 7ms/step - loss: 0.9782
Epoch 4/50
1/1 [==============================] - 0s 7ms/step - loss: 0.9782
Epoch 5/50
1/1 [==============================] - 0s 7ms/step - loss: 0.9782
Epoch 6/50
1/1 [==============================] - 0s 6ms/step - loss: 0.9782
Epoch 7/50
1/1 [==============================] - 0s 7ms/step - loss: 0.9782
Epoch 8/50
1/1 [==============================] - 0s 6ms/step - loss: 0.9782
Epoch 9/50
1/1 [==============================] - 0s 7ms/step - loss: 0.9782
Epoch 10/50
1/1 [==============================] - 0s 6ms/step - loss: 0.9782
Epoch 11/50
1/1 [==============================] - 0s 8ms/step - loss: 0.9782
Epoch 12/50
1/1 [==============================] - 0s 7ms/step - loss: 0.9782
Epoch 13/50
1/1 [==============================] - 0s 6ms/step - loss: 0.

## Test

In [43]:
decoder = load_model('../models/decoder.h5')

In [44]:
example_geotag = np.array([X[0]])

In [45]:
latent = mlp.predict(example_geotag)
audio = decoder.predict(latent)

1/1 [==============================] - 0s 63ms/step


In [46]:
ipd.display(ipd.Audio(data=audio, rate=44100))

In [22]:
mlp.save('../models/mlp.h5')